In [1]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [2]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [3]:
fea_number=7
data=pd.read_csv('./BayesianRidge_Pre_0/Input_A6_024.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
y=data.Predict
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_A6_022', 'Input_A6_023', 'Input_A5_024', 'Input_A5_022',
       'Input_A5_023', 'Input_A4_024', 'Input_A4_022'], dtype=object)

## Linear Regression Baseline

In [7]:
lr_rmse_ave_fea(skew_data,7)

train_rmse: 0.0023932372896607094
test_rmse: 0.002902466068294067
test_r2: 0.9479807856847199


## Lasso

In [5]:
laso_rmse_ave(skew_data[title],y,5e-05)

test_rmse_ave: 0.002858629886444398
[0.0022698036385402605, 0.0026546833454194245, 0.004410207615975337, 0.0025281182625454613, 0.002642067728980637, 0.0028450253357848038, 0.0021151745802722947, 0.002027588105765336, 0.002933624158560253, 0.004160006092600172]


test_r2_ave: 0.9499885212469665
[0.9642879587892206, 0.9548010681206917, 0.8860537267354931, 0.967925264694656, 0.9680361113595436, 0.9614937583590666, 0.9639196491529832, 0.9687003476063686, 0.9483272412105466, 0.9163400864410954]


## ElasticNet

In [6]:
ElasticNet_rmse_ave(skew_data[title],y,[0.008],0.01)

test_rmse_ave: 0.002929450984140303
[0.002277065011807448, 0.003040880814779033, 0.004298547483106992, 0.0026067186890523296, 0.002628859786139357, 0.0029324636452928376, 0.0022889907356514637, 0.002207130616465935, 0.0030440251141509137, 0.0039698279449567235]


test_r2_ave: 0.947868570160925
[0.9640590991184206, 0.9406936059744937, 0.8917505962743009, 0.9658998224222778, 0.9683548934694092, 0.9590905040274521, 0.9577461404201756, 0.9629117693677869, 0.9443648626536585, 0.923814407881274]


## SVR

In [8]:
model=SVR(kernel='linear', C=1.1, gamma= 1e-07, epsilon= 0)
svr_rmse_ave(skew_data,7,model)

train_rmse: 0.002494766253742296
test_rmse: 0.0029676558528763256
test_r2: 0.9455290284090244


## XGB

In [9]:
best_xgb_model =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.12,
              booster = 'gbtree', 
              n_estimators = 140, 
              max_depth = 3, 
              min_child_weight = 2,
              seed = 42,
              gamma = 0,
              subsample = 0.8,
              colsample_bytree = 0.7,
              reg_alpha =  0,
              reg_lambda = 1)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.0035365228166043794
[0.0029249752911252986, 0.003577783758349782, 0.004996112482970316, 0.0030425243381591183, 0.003798613888601857, 0.0032827121373231404, 0.002577093196136331, 0.0034379342427416637, 0.00333061871250566, 0.004396860118130631]


test_r2_ave: 0.9244963850321308
[0.9406962028248715, 0.9179023174798346, 0.85376663591797, 0.9535445756794102, 0.9339273052743419, 0.9487345873932131, 0.9464402472453857, 0.9100139526082397, 0.9333956586942103, 0.9065423672038302]


## RandomForest

In [10]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 70,
            max_depth = 4,
            min_samples_split = 4,
            max_leaf_nodes = 8,
            min_samples_leaf =4,
            random_state = 42) 
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.0037110227960922893
[0.0028879054002281745, 0.004109467475831522, 0.005306374592196106, 0.0029308081035650346, 0.003722203379855058, 0.0035235326719868635, 0.0029075626640337293, 0.003504785901096578, 0.003382251182373817, 0.0048353365897560105]


test_r2_ave: 0.9159898999416928
[0.9421898596077684, 0.8916886879828615, 0.8350402971876476, 0.9568934687775296, 0.9365587232922888, 0.9409370056274383, 0.9318232206025572, 0.9064803159705186, 0.9313146023804945, 0.8869728179878225]


In [11]:
lr = LinearRegression()
lasso_mod=Lasso(alpha=5e-05)
elastic_mod=ElasticNet(alpha=[0.008], l1_ratio=0.01)
svr_mod=SVR(kernel='linear', C=1.1, gamma= 1e-07, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.12,
              booster = 'gbtree', 
              n_estimators = 140, 
              max_depth = 3, 
              min_child_weight = 2,
              seed = 42,
              gamma = 0,
              subsample = 0.8,
              colsample_bytree = 0.7,
              reg_alpha =  0,
              reg_lambda = 1)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 70,
            max_depth = 4,
            min_samples_split = 4,
            max_leaf_nodes = 8,
            min_samples_leaf =4,
            random_state = 42) 
vote_mod = VotingRegressor([ ('Lasso', lasso_mod), ('SVR', svr_mod),('Elastic', elastic_mod), ('Linear', lr),
                            ('XGBRegressor', xgb_mod),('RandomForest', random_mod)])
# 
# vote_mod = VotingRegressor([ ('SVR', svr_mod), ('Linear', lr)
#                             ,('RandomForest', random_mod)])
voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.0029389200477266154
[0.0023200479968436146, 0.0028351086494880972, 0.004525092890200137, 0.0024937218926521666, 0.002802440239438371, 0.0028464632290841805, 0.0021920056528860017, 0.002241072889962297, 0.0029229323090666, 0.0042103147276446855]


test_r2_ave: 0.9471483310994664
[0.9626894167951272, 0.948448401614436, 0.8800398352962635, 0.9687921143981214, 0.9640379503330954, 0.9614548259405933, 0.961250896917063, 0.961762278246104, 0.9487032066063124, 0.9143043848475468]


In [12]:

stack_mod = StackingRegressor(regressors=[vote_mod, svr_mod,elastic_mod,lr], 
                           meta_regressor=lasso_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.0028586298864443976
[0.0022698036385402605, 0.0026546833454194245, 0.004410207615975336, 0.0025281182625454613, 0.002642067728980637, 0.0028450253357848033, 0.0021151745802722947, 0.002027588105765336, 0.0029336241585602527, 0.004160006092600172]


test_r2_ave: 0.9499885212469665
[0.9642879587892206, 0.9548010681206917, 0.8860537267354931, 0.967925264694656, 0.9680361113595436, 0.9614937583590666, 0.9639196491529832, 0.9687003476063686, 0.9483272412105466, 0.9163400864410953]


In [14]:
ave(skew_data[title],y,0.65,stack_mod,0.1,vote_mod,0.25,lasso_mod)

test_rmse_ave: 0.002861808889893771
[0.00227102925259986, 0.002668417341119121, 0.004418221710144785, 0.002519615209856076, 0.0026476386017222557, 0.0028377132812216657, 0.0021209421029632835, 0.0020422699382138154, 0.0029290731615075538, 0.004163168299589297]


test_r2_ave: 0.9498622401473025
[0.9642493818868227, 0.9543321853124507, 0.8856392311191128, 0.96814066166723, 0.9679011757685114, 0.9616914352762457, 0.963722617708691, 0.9682454228517617, 0.9484874390775972, 0.9162128508046018]
